## 캐시(Cache)
#### 시간과 비용이 많이 들거나 자주 엑세스 하는 계산 결과를 저장해둠으로써 웹이나 프로그램이 실행될때마다 다시 계산할 필요가 없도록 효율화하여 프로그램의 성능을 높이는데 도움을 주는 메커니즘(임시저장소의 개념)
- Streamlit에서는 @st.cache_data, @st.cache_resource 두 가지 코드로 사용자 정의 함수에 데코레이션 된 형태로 적용
  - cache_data : 자료나 데이터 구조를 반환해주는 함수에 적용. ex) DataFrame, array, list 등
  - cache_resource : 데이터가 아닌 상대적으로 무거운 리소스를 반환하는 함수에 적용. ex) 인공지능 모델, DB연결 등
- 데코레이션 된 함수가 호출되면 Streamlit은 이전과 동일한 매개변수로 호출되었는지를 확인
- 동일한 매개변수라면 미리 계산된 결과를 바로 출력시켜주며, 매개변수나 코드가 변경되거나 세션이 종료된다면 저장된 캐시를 삭제하고 함수를 재실행
- 캐시는 Streamlit 애플리케이션을 실행하는 시스템의 RAM에 임시 보관
- 작업에 시간이 많이 걸릴 수 있는 데이터 과학 및 인공지능 분야에서 정보를 저장해두고 활용하기에 유용한 기능

#### 아래와 같은 비효율적인 상황을 피하기 위해 사용
- 시간이 많이 소요되는 함수가 불필요하게 반복 실행되는 상황
- 한 번 만들어 두면 되는 객체가 새로고침 할 때마다 재생성되는 상황

In [8]:
%%writefile module/myApp20.py

import streamlit as st
import pandas as pd
import time

st.title('Streamlit 캐시 실습')

# 사용자 정의 함수에 캐시 선언(데코레이션)
@st.cache_data
def cal_sum(a, b) :
    time.sleep(5)
    return a+b

# 숫자 2개 입력 받기
a = st.number_input('첫번째 숫자 입력', min_value=0, max_value=10, step=1)
b = st.number_input('두번째 숫자 입력', min_value=0, max_value=10, step=1)

# 버튼으로 함수 계산 실행
button_start = st.button('계산 실행😊', key=1)
if button_start :
    result = cal_sum(a, b)
    st.write(f'계산결과 >> {result}')

# 캐시 삭제 버튼 추가
button_clear = st.button('캐시 삭제✨', key=2)
if button_clear :
    st.cache_data.clear()    # 전체 캐시 삭제해주는 명령
    st.write('캐시 삭제 완료!')
''
'___'
''
@st.cache_data
def get_data(age) :
    myDict = {'이름':'성우', '나이':age, '성별':'남'}
    df = pd.DataFrame(myDict, index=['회원정보'])
    time.sleep(3)
    return df
''
myAge = st.slider('나이 입력', 20, 40, 30)
''
button_age = st.button('입력 및 실행👌', key=3)
if button_age :
    result = get_data(myAge)
    st.write(result)

# 특정 함수만 캐시 초기화를 하고 싶다면?
 # 위의 cache_data.clear()가 적용된 버튼을 한번누르면 웹 페이지 내의 모든 캐시가 초기화 되기 때문에 불편할 수 있음
# 버튼 함수 내에 '사용자정의 함수명.clear'로 지정 후 아래 조건문에서 .clear()로 실행
# on_click은 이 함수만 가져와서 지우겠다
button_clear2 = st.button('캐시 삭제✨', key=4, on_click=get_data.clear)        
if button_clear2 :
    get_data.clear()
    st.write('캐시 삭제 완료!!')

Overwriting module/myApp20.py


### 폼 (Form)
#### 위젯들은 그룹으로 묶어 여러 입력값을 받아서 저장해두며 한 번에 제출(입력)하는 용도로 활용
- 목적 : 개별 위젯 값이 변경될 때마다 전체 웹사이트에 재실행되는 비효율을 방지하고 여러 입력 위젯들을 그룹화 해서 관리
- 폼에서는 st.button이나 st.download_button은 사용할 수 없음
- 단 st.form_submit_button(최종제출버튼)은 반드시 하나 이상 포함되어야 함
- 최종제출버튼을 누르기 전에는 폼에 포함된 다른 값들의 변화가 일어나지 않음
- 폼 내부에 다른 폼이 들어갈 수 없음

In [3]:
%%writefile module/myApp21.py

import streamlit as st
import numpy as np
import pandas as pd
import time

# 사용자 정의 함수에 캐시 선언(데코레이션)
@st.cache_data
def get_data(name, age, gender) :
    myDict = {'이름':name, '나이':age, '성별':gender}
    df = pd.DataFrame(myDict, index=['회원정보'])
    time.sleep(3)
    return df

# 1. 폼 미적용
st.subheader('폼 미적용 상태')
name1 = st.text_input("What's your name?")
age1 = st.slider("How old are you?", 20, 40, 30)
gender1 = st.radio("What is your gender?", ['남','여'])
st.write(get_data(name1, age1, gender1))
''
'___'
''

# 2. 폼 적용
st.subheader('폼 적용 상태')
# clear_on_submit=True : 제출버튼을 누르면 위젯에 입력한 값들이 초기화 상태로 변경(False는 그대로 놔둠)
with st.form(key='form1', clear_on_submit=True) :
    name2 = st.text_input("What's your name?")
    age2 = st.slider("How old are you?", 20, 40, 30)
    gender2 = st.radio("What is your gender?", ['남','여'])
    st.write(get_data(name1, age1, gender1))

    submit_button = st.form_submit_button('제출하기!')

    if submit_button :
        st.write(get_data(name2, age2, gender2))

Overwriting module/myApp21.py
